# Lesson 4: Chains
https://learn.deeplearning.ai/langchain/lesson/4/chains

- LLMChain
- Sequential Chains
  - SimpleSequentialChain
  - SequentialChain
- Router Chain

In [2]:
import os
import warnings

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from langchain.chains import LLMChain, SequentialChain, SimpleSequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

warnings.filterwarnings("ignore")

In [3]:
load_dotenv(find_dotenv())

try:
    os.environ["OPENAI_API_KEY"]
except Exception:
    raise Exception("No Dashscope key found!")

In [4]:
CHAT_GPT_MODEL = "glm-4"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

In [5]:
df = pd.read_csv("Data.csv")

df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...


### LLMChain

In [6]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0.01, base_url=BASE_URL)

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
product = "Queen Size Sheet Set"

In [10]:
chain.run(product=product)

"Choosing a name for a company that specializes in manufacturing Queen Size Sheet Sets can be a blend of creativity and marketing strategy. The name should ideally be memorable, reflective of the brand's values, and give customers a clear idea of what the company offers. Here are a few suggestions:\n\n1. Queenly Comfort Co.\n2. Sheets of Elegance\n3. Royal Slumber Sheets\n4. Queen's Quilted Haven\n5. Luxe Linen Queens\n6. Pillowtop Palace Sheets\n7. Dreamy Queen Sheets\n8. Regal Rest Linens\n9. Queen Quilt Company\n10. Serene Sleep Sheets\n\nWhen choosing a name, consider the following:\n\n- **Relevance:** The name should be relevant to the product being sold.\n- **Memorability:** A good name is easy to remember, pronounce, and spell.\n- **Originality:** It should be unique to avoid confusion with other brands.\n- **Scalability:** If you plan to expand your product line in the future, the name should not limit your offerings to just Queen Size Sheet Sets.\n- **International Considerati

### SimpleSequentialChain

In [11]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0.01, base_url=BASE_URL)

In [12]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True,
)

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Choosing a name for a company that specializes in manufacturing Queen Size Sheet Sets can be a blend of creativity and marketing strategy. The name should ideally be memorable, reflective of the brand's values, and give customers a clear idea of what the company offers. Here are a few suggestions:

1. Queenly Comfort Co.
2. Sheets of Elegance
3. Royal Slumber Sheets
4. Queen's Quilted Haven
5. Luxe Linen Queens
6. Pillowtop Palace Sheets
7. Dreamy Queen Sheets
8. Regal Rest Linens
9. Queen Quilt Company
10. Serene Sleep Sheets

When choosing a name, consider the following:

- **Relevance:** The name should be relevant to the product being sold.
- **Memorability:** A good name is easy to remember and stands out from competitors.
- **Originality:** It's important to have a unique name that isn't already trademarked by another company.
- **Simplicity:** Shorter names are generally easier to remember and can be more effective.
- **Brandabilit

'"Crafting luxury Queen Size Sheet Sets, \'Royal Slumber Sheets\' offers regal comfort for serene sleep."'

### SequentialChain

In [16]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0.01, base_url=BASE_URL)

In [17]:
first_prompt = ChatPromptTemplate.from_template(
    template="Translate the following review to english:\n\n{Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

second_prompt = ChatPromptTemplate.from_template(
    template="""Can you summarize the following review \
        in 1 sentence:\n\n{English_Review}"""
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template(
    template="What language is the following review:\n\n{Review}"
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [18]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [19]:
review = df.Review[5]

overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...Vieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold up, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer finds the product's taste mediocre and the foam disappointing, suspecting it might be an old batch or counterfeit compared to the better-tasting ones purchased in stores. \n\nOne-sentence summary: The reviewer deems the product's taste subpar and foam unstable, questioning if it's an old batch or fake compared to the superior version from stores.",
 'followup_message': 'Suite à votre résumé, voici la réponse en français :\n\n"Je suis d\'accord avec votre évaluation concernant la saveur médiocre et l\'instabilité de la mousse. Ces incohérences dans la qualité suscitent des doutes quant à la fraîcheur du produit ou

### Router Chain

In [20]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, \
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [21]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "history",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [22]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0.01, base_url=BASE_URL)

In [23]:
destination_chains: dict[str, LLMChain] = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [24]:
destination_chains

{'physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x72dfe58c5d80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x72dfe58f11e0>, model_name='glm-4', temperature=0.01, openai_api_key=SecretStr('**********'), openai_api_base='https://open.bigmodel.cn/api/paas/v4/', openai_proxy='')),
 'math': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a very good mathematician. You are great at answering math questio

In [25]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

In [26]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nhistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [27]:
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [28]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str,
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [30]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [31]:
chain.run("What is black boday radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Blackbody radiation is the electromagnetic radiation emitted by an idealized object known as a blackbody. A blackbody is an object that absorbs all incident radiation, regardless of frequency or angle of incidence, and does not reflect or transmit any of it. Instead, it re-emits radiation at a rate that depends solely on its temperature.\n\nThe radiation emitted by a blackbody covers all wavelengths and intensities, creating a continuous spectrum that peaks at a wavelength corresponding to the temperature of the object. At room temperature, most of the radiation is in the infrared part of the spectrum. As the temperature increases, the peak of the spectrum shifts towards the visible and then to the ultraviolet range.\n\nThe distribution of this radiation with respect to wavelength is described by Planck's law, which states that the spectral radiance of a blackbody at a given temperature is a function of the wavelength and temperature, and is proportional to the Wien's displacement law

In [32]:
chain.run("What is 2 + 2?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


'Thank you for the compliment! Let\'s break down the problem you\'ve presented:\n\nThe question is: What is 2 + 2?\n\nThis is a simple addition problem involving two numbers, 2 and 2. To solve it, we add the component parts:\n\n2 (first number) + 2 (second number) = 4\n\nSo, the answer to the broader question "What is 2 + 2?" is 4.'

In [33]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

In [34]:
chain.run("Who was the first emperor in China's history")



> Entering new MultiPromptChain chain...
history: {'input': "Who was the first emperor in China's history?"}
> Finished chain.


"The first emperor in China's history was Qin Shi Huang, who ruled from 221 to 210 BCE. He is著名的 for unifying the Warring States, which were various regions that had been in conflict with each other for centuries, and establishing the Qin Dynasty. Qin Shi Huang's reign was marked by significant administrative, economic, and cultural reforms, including the standardization of writing, weights, measures, and the construction of the first version of the Great Wall. He is also known for his mausoleum, which includes the famous Terracotta Army. His efforts to centralize and consolidate power laid the groundwork for the imperial system that would continue in China for over two millennia."